In [1]:
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
# location of the local and remote stats
moment = "mean"
local_stat = f"ocn.fvstatistics.l10.1993-2019/{moment}_by_level.zarr"
gcp_stat = f"gcs://noaa-ufs-gefsv13replay/ufs-hr1/0.25-degree-subsampled/06h-freq/zarr/mom6.fvstatistics.l10.1993-2019/{moment}_by_level.zarr"

In [3]:
# open local stats
f_local = xr.open_zarr(local_stat)
f_local

<xarray.Dataset> Size: 309kB
Dimensions:       (z_l: 10, lat: 192, lon: 384, time: 37985, z_i: 11)
Coordinates:
  * lat           (lat) float64 2kB 89.82 88.89 87.95 ... -87.25 -88.18 -89.12
  * lon           (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * time          (time) datetime64[ns] 304kB 1993-12-31T18:00:00 ... 2019-12...
  * z_i           (z_i) float64 88B 0.0 1.031 4.498 10.8 ... 190.2 352.3 483.1
  * z_l           (z_l) float64 80B 0.5154 2.765 7.648 ... 152.1 271.2 417.7
Data variables:
    landsea_mask  (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>
    so            (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>

In [4]:
# open remote 
open_kwargs = dict(storage_options={"token":"anon"})
f_gcp = xr.open_zarr(gcp_stat, **open_kwargs)
f_gcp

<xarray.Dataset> Size: 580B
Dimensions:  (z_l: 10, z_i: 11)
Coordinates:
  * z_i      (z_i) float64 88B 0.0 1.031 4.498 10.8 ... 114.0 190.2 352.3 483.1
  * z_l      (z_l) float64 80B 0.5154 2.765 7.648 15.95 ... 152.1 271.2 417.7
Data variables:
    LW       float32 4B ...
    SSH      float32 4B ...
    SW       float32 4B ...
    dz       (z_l) float64 80B dask.array<chunksize=(10,), meta=np.ndarray>
    so       (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>
    temp     (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>
    uo       (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>
    vo       (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>

In [5]:
landsea_mask_stats = f_local.landsea_mask.copy(deep=True)
landsea_mask_stats

<xarray.DataArray 'landsea_mask' (z_l: 10)> Size: 80B
dask.array<open_dataset-landsea_mask, shape=(10,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * z_l      (z_l) float64 80B 0.5154 2.765 7.648 15.95 ... 152.1 271.2 417.7
Attributes:
    description:  standard deviation over ['lat', 'lon']
    long_name:    land-sea mask (land=0, sea/ice=1)
    units:        None

In [6]:
# convert to dataset and copy the original global attributes
landsea_mask_ds = landsea_mask_stats.to_dataset(name="landsea_mask",)
landsea_mask_ds.attrs = f_gcp.attrs.copy()
landsea_mask_ds

<xarray.Dataset> Size: 160B
Dimensions:       (z_l: 10)
Coordinates:
  * z_l           (z_l) float64 80B 0.5154 2.765 7.648 ... 152.1 271.2 417.7
Data variables:
    landsea_mask  (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>

In [8]:
# Append to existing zarr stores: remove capitals and specials in their equivalents in lower case
landsea_mask_ds.to_zarr(gcp_stat, mode="a", consolidated=True,
                        storage_options={"token":"/hOmE/nAgArw@l/.gcs/replay-service-account.json"})